In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bank-transaction-dataset-for-fraud-detection/bank_transactions_data_2.csv


Load the bank transactions data from the provided CSV file.

In [3]:
try:
    df = pd.read_csv('/kaggle/input/bank-transaction-dataset-for-fraud-detection/bank_transactions_data_2.csv')
    display(df.head())
except FileNotFoundError:
    print("Error: 'bank_transactions_data_2.csv' not found. Please check the file path.")
except pd.errors.ParserError:
    print("Error: Could not parse the CSV file. Please check the file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39


Explore the loaded data to identify the target variable for fraud detection.

In [4]:
# Check for common target variable names (case-insensitive).
target_candidates = ['fraud', 'is_fraud', 'fraudulent', 'is_fraudulent', 'class']
for col in df.columns:
    if col.lower() in target_candidates:
        print(f"Potential target variable found: {col}")
        break
else:
    # No obvious target column found by name, inspect data types and distributions.
    print("No obvious target column found by name. Inspecting data types and distributions.")
    for col in df.columns:
        print(f"Column: {col}, Data Type: {df[col].dtype}, Unique Values: {df[col].nunique()}")

    # Look for binary columns that might indicate fraud.
    for col in df.columns:
        if df[col].nunique() == 2:
            print(f"Potentially binary column: {col}, Unique Values: {df[col].unique()}")

No obvious target column found by name. Inspecting data types and distributions.
Column: TransactionID, Data Type: object, Unique Values: 2512
Column: AccountID, Data Type: object, Unique Values: 495
Column: TransactionAmount, Data Type: float64, Unique Values: 2455
Column: TransactionDate, Data Type: object, Unique Values: 2512
Column: TransactionType, Data Type: object, Unique Values: 2
Column: Location, Data Type: object, Unique Values: 43
Column: DeviceID, Data Type: object, Unique Values: 681
Column: IP Address, Data Type: object, Unique Values: 592
Column: MerchantID, Data Type: object, Unique Values: 100
Column: Channel, Data Type: object, Unique Values: 3
Column: CustomerAge, Data Type: int64, Unique Values: 63
Column: CustomerOccupation, Data Type: object, Unique Values: 4
Column: TransactionDuration, Data Type: int64, Unique Values: 288
Column: LoginAttempts, Data Type: int64, Unique Values: 5
Column: AccountBalance, Data Type: float64, Unique Values: 2510
Column: PreviousTra

The provided dataset lacks a clearly labeled target variable for fraud detection. The analysis explored several columns, but none were definitively identified as indicating fraud. TransactionType was considered due to its binary nature (Debit/Credit), but no clear connection to fraudulent activity was found.
Data Analysis Key Findings

    No column name directly suggests fraud (e.g., 'fraud', 'is_fraud').
    The analysis focused on TransactionType due to its binary nature, but its frequency counts did not reveal any obvious connection to fraudulent activity.
    Visualizations of transaction amounts and other numerical features (CustomerAge, TransactionDuration, LoginAttempts, AccountBalance) did not reveal any clear outliers or anomalies indicative of fraud.
    The analysis of the distribution of TransactionType and its relationship with numerical features like TransactionAmount, CustomerAge, etc., did not reveal any strong patterns suggesting fraudulent activity.

Insights or Next Steps

    Further investigation with domain expertise is needed to identify a suitable target variable or relevant features for fraud detection.
    Additional data, such as a dedicated fraud label or features related to transaction location and device information, might be required for a meaningful fraud detection analysis.


In [5]:
num_columns = df.shape[1]
print(f"The DataFrame has {num_columns} columns.")

The DataFrame has 16 columns.


Since there is no target column, let us evaluate using unsupervised algorithms. We know we can use DBScan. But before that we shall use  isolation forest as it is much faster.

Exploring dimentionality reduction:
With 16 columns, we likely don't need to perform dimensionality reduction techniques like PCA or t-SNE before applying Isolation Forest. Isolation Forest is generally quite effective even with a moderate number of features.

Let us start with the 16 features and apply Isolation Forest directly. This will give us a baseline performance and allow us to see how the model behaves with your current data. We can explore dimensionality reduction later if needed. 

We shall do the problem with isolation forest and as and when the rows are identified as fraudulent or regular, the label will be assigned to that row. Thus at the end we shall have an enhanced dataset with an additional column called "is_fraudulent" and the values will be 1 for yes and 0 for no.

In [6]:
from sklearn.ensemble import IsolationForest
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assuming our DataFrame is named 'df' and is already loaded

# Identify numerical and categorical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object', 'category']).columns

# Create a preprocessor to handle numerical and categorical features
# Numerical features will be passed through (or we could add scaling)
# Categorical features will be one-hot encoded
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # handle_unknown='ignore' handles new categories in test data if applicable
    ])

# Create a pipeline that first preprocesses the data and then applies Isolation Forest
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('isolationforest', IsolationForest(n_estimators=100,
                                                          contamination='auto',
                                                          random_state=42))])

# Fit the model on the entire DataFrame (the pipeline handles preprocessing)
model.fit(df)

# Predict anomaly labels
# The predict method of the pipeline will run the data through the preprocessor first
predictions = model.predict(df)

# we work on a copy
df_augmented = df.copy()

# Add the new column to the DataFrame
# Map predictions (-1, 1) to labels (1, 0)
df_augmented['is_fraudulent'] = [1 if pred == -1 else 0 for pred in predictions]

# Display the first few rows with the new column
print("DataFrame with 'is_fraudulent' column:")
display(df_augmented.head())

# we can also check the value counts of the new column
print("\nValue counts for 'is_fraudulent':")
print(df_augmented['is_fraudulent'].value_counts())

DataFrame with 'is_fraudulent' column:


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,is_fraudulent
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08,0
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35,0
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04,0
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06,0
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39,0



Value counts for 'is_fraudulent':
is_fraudulent
0    2512
Name: count, dtype: int64


The Isolation Forest model on contanination="auto" did n't find any anomaly, which is possible. However if from our prior experiecne if we know that 2% of the transaction are fraudulent, then we can use the below code to find which of these rows are most likely to be fraudulent. This is not to say that they are fraudulent, but are definitely the top 2% most suspicious.

In [7]:
# Create a pipeline that first preprocesses the data and then applies Isolation Forest
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('isolationforest', IsolationForest(n_estimators=100,
                                                          contamination=0.02,
                                                          random_state=42))])

# Fit the model on the entire DataFrame (the pipeline handles preprocessing)
model.fit(df)

# Predict anomaly labels
# The predict method of the pipeline will run the data through the preprocessor first
predictions = model.predict(df)

# we work on a copy
df_augmented2 = df.copy()

# Add the new column to the DataFrame
# Map predictions (-1, 1) to labels (1, 0)
df_augmented2['is_fraudulent'] = [1 if pred == -1 else 0 for pred in predictions]

# Display the first few rows with the new column
print("DataFrame with 'is_fraudulent' column:")
display(df_augmented2.head())

# we can also check the value counts of the new column
print("\nValue counts for 'is_fraudulent':")
print(df_augmented2['is_fraudulent'].value_counts())

DataFrame with 'is_fraudulent' column:


,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,is_fraudulent
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70,Doctor,81,1,5112.21,2024-11-04 08:08:08,0
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68,Doctor,141,1,13758.91,2024-11-04 08:09:35,0
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19,Student,56,1,1122.35,2024-11-04 08:07:04,0
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26,Student,25,1,8569.06,2024-11-04 08:09:06,0
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26,Student,198,1,7429.40,2024-11-04 08:06:39,0



Value counts for 'is_fraudulent':
is_fraudulent
0    2461
1      51
Name: count, dtype: int64


In [8]:
# print the top 10 suspected ones
df_augmented2[df_augmented2['is_fraudulent'] == 1].head(10)

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,is_fraudulent
10,TX000011,AC00150,17.78,2023-03-14 16:46:10,Credit,Albuquerque,D000205,213.15.9.253,M073,Online,52,Engineer,59,1,7599.52,2024-11-04 08:06:45,1
81,TX000082,AC00445,345.39,2023-10-23 17:13:57,Credit,Phoenix,D000097,131.41.45.13,M081,Online,58,Engineer,80,1,6451.96,2024-11-04 08:08:16,1
91,TX000092,AC00310,223.85,2023-10-02 16:36:10,Debit,Kansas City,D000481,133.223.159.151,M009,ATM,53,Engineer,55,4,2929.91,2024-11-04 08:08:01,1
96,TX000097,AC00348,858.63,2023-04-18 17:27:23,Credit,San Francisco,D000019,175.230.206.232,M002,Branch,27,Student,185,1,1872.96,2024-11-04 08:10:47,1
99,TX000100,AC00178,115.02,2023-09-11 16:19:04,Debit,San Antonio,D000528,134.222.122.238,M004,Branch,37,Doctor,175,1,7643.66,2024-11-04 08:08:38,1
140,TX000141,AC00403,6.00,2023-10-23 18:38:15,Debit,Chicago,D000556,18.72.115.111,M090,Online,30,Doctor,246,1,5442.76,2024-11-04 08:09:43,1
150,TX000151,AC00072,69.40,2023-07-24 18:08:43,Debit,Virginia Beach,D000413,216.151.64.40,M073,Online,80,Retired,113,1,6452.77,2024-11-04 08:09:04,1
153,TX000154,AC00442,496.03,2023-11-03 16:45:52,Debit,San Francisco,D000070,210.160.90.52,M009,Branch,61,Doctor,265,1,12904.66,2024-11-04 08:09:41,1
208,TX000209,AC00453,521.15,2023-02-17 18:16:05,Debit,Kansas City,D000108,166.7.230.111,M084,ATM,54,Doctor,214,1,9442.47,2024-11-04 08:07:31,1
359,TX000360,AC00165,626.90,2023-10-10 16:42:09,Debit,Chicago,D000686,59.12.96.11,M017,ATM,23,Student,146,1,587.11,2024-11-04 08:11:30,1
